In [2]:
#import necessary package and collect initial variables
import openmdao.api as om
import pandas as pd
import numpy as np

In [44]:
#define discipline group, get constants and call analyses 
class sys(om.Group):
    def initialize(self):
        
        #read excel constants
        initial_vars = pd.read_excel('constants.xlsx')
        
        #sort excel constants and take only name, value and description columns
        all_sys_vars = np.where(initial_vars['sys'] == True, [initial_vars['variable name'], initial_vars['value'], initial_vars['description']], None)
        all_sys_vars = pd.DataFrame(all_sys_vars).dropna(axis = 1).transpose()

        #put variables into openMDAO format to be available to use in calcs
        i=0
        for variable in all_sys_vars[0]:
            value = all_sys_vars.iloc[i][1]
            description = all_sys_vars.iloc[i][2]
            self.options.declare(variable, default = value, desc = description)
            i+=1
        i=0
        
        
    def setup(self):
        #set analysis names and what inputs they take and give
        self.add_subsystem('fuel_masses', fuel_masses()) #add 'promotes =' to promote variables, otherwise connect them directly
        

    #optional measure to promote all variables from a component
    #def configure(self):
        #self.promotes('fuel_masses',any=['*'])
        


In [16]:
class fuel_masses(om.ExplicitComponent):
        
    def setup(self):

        self.add_output('sys_CG')
    
    def compute(self,inputs,outputs):
        Is = self.options['']
        
        outputs['sys_CG'] = 2*Is
